In [1]:
import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import to_timestamp, to_date, lit, sort_array, collect_list, col, udf
import pyspark.sql.functions as f
import json
from pyspark.sql.functions import udf
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.types import DateType
spark.conf.set("spark.sql.session.timeZone", "UTC+08:00")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1610938585457_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# load data

In [2]:
Timer1 = time.time()
df_item = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load('s3://douyu-datalab/tmp_recom_item_feature_1d_20201225.csv')
#fillna
df_item = df_item.fillna(0)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_user = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load('s3://douyu-datalab/tmp_recom_user_feature_1d_20201225.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# uid: 
# room_id: 
# watch_time: (ms)
# watch_cnt: click times per day
df_weight = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load('s3://douyu-datalab/tmp_recom_weight_data_1d_20201225.csv')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# uid: 
# room_id: 
# action_code: show_recommend|click_recommend
# user_pos: position in push list(after sort?)
# ct_pt: platform(ios/android)
# src: recall source
df_action_code = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load('s3://douyu-datalab/tmp_recom_action_code_1d_20201225.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# process label

In [6]:
df_weight.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------+---------+
|                 uid|             room_id| watch_time|watch_cnt|
+--------------------+--------------------+-----------+---------+
|d41d8cd98f00b204e...|5a6caffa78e4b35e7...|  237429069|      681|
|d41d8cd98f00b204e...|1831788bc34373afc...|      35561|        3|
|d41d8cd98f00b204e...|372d0439bee1dcd7f...|    6851946|        2|
|d41d8cd98f00b204e...|61400900a16c0e3c9...|    2999402|        1|
|d41d8cd98f00b204e...|70c72246f05283b61...|    1529807|        5|
|d41d8cd98f00b204e...|f61846e56bf93f732...|      19946|        3|
|d41d8cd98f00b204e...|e36ed67ab14674311...|     249368|       24|
|d41d8cd98f00b204e...|3964989eb0b40a372...|52330972945|   285569|
|d41d8cd98f00b204e...|3efc81de877967a7a...|      11565|        4|
|d41d8cd98f00b204e...|67e31581af871253b...|    1472517|       25|
|d41d8cd98f00b204e...|cb363709e9f4adfab...|   39286860|       11|
|d41d8cd98f00b204e...|e748c45cceb1ad8e5...|     725005|        3|
|d41d8cd98

In [7]:
df_action_code.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+--------------------+--------------------+--------+-------+----+
|     dateline|                 uid|             room_id|         action_code|user_pos|  ct_pt| src|
+-------------+--------------------+--------------------+--------------------+--------+-------+----+
|1608552932348|5a0d2e6f67148a4bd...|a880f447e08c316ea...|show_recommend_re...|       2|android|9810|
|1608520873257|3a0ebc104c8c6faf9...|38b3eff8baf566274...|show_recommend_re...|       1|    ios|9831|
|1608562029075|7f894553142e46cf3...|ae8b5aa26a3ae3161...|show_recommend_re...|       3|    ios|1005|
|1608565452016|8d9fc5944d88166c1...|6d70d6bd21e71385f...|show_recommend_re...|      59|android|   0|
|1608565345690|8d9fc5944d88166c1...|3ed304a448828ee2c...|show_recommend_re...|     172|android|1004|
|1608518783435|b57629c6aaffeaed1...|ae9fb56eaa2096c74...|show_recommend_re...|       3|android|1005|
|1608531578847|ac6ddf1e3c74ca94c...|89209f444391e9c72...|click_recommend_r...|       2|    

In [8]:
#convert to datetime
df_action_code = df_action_code.withColumn('datetamp', (df_action_code.dateline/1e3).cast(IntegerType()))
df_action_code = df_action_code.withColumn('datetamp2', from_unixtime(df_action_code.datetamp))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def convert_label(x):
    if x=='click_recommend_recom_hot_room':
        return 1
    else:
        return 0

#use age, owner_level,sex,fans
udf_func_convert = udf(lambda x: convert_label(x),returnType=IntegerType())
df_action_code = df_action_code.withColumn("label",udf_func_convert(df_action_code["action_code"]))
#group by userid, roomid
#此处将曝光未点击的记录，直接当成曝光点击的前置部分？？？如果两者时间差别比较大呢？
df_action_code_agg = df_action_code.groupby(df_action_code.uid,df_action_code.room_id).\
agg(f.max('label').alias("label_click"))
#left join watch to filter conversion
df_res = df_action_code_agg.join(df_weight,on=["room_id","uid"],how='left')
#convert watchtime to second
df_res2 = df_res.withColumn("time_min", df_res["watch_time"]/60000)

df_res2 = df_res2.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def convert_cvr_label(label_click,time_min):
    if int(label_click)==1 and float(time_min)>5:
        return 1
    else:
        return 0

    
udf_func_convert = udf(lambda x,y: convert_cvr_label(x,y),returnType=IntegerType())
df_res2 = df_res2.withColumn("cvr_label",udf_func_convert(df_res2["label_click"],df_res2["time_min"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
def convert_ctr_label(x,y):
    if int(x)==1 and float(y)>2:
        return 1
    else:
        return 0

udf_func_convert = udf(lambda x,y: convert_ctr_label(x,y),returnType=IntegerType())
df_res2 = df_res2.withColumn("ctr_label",udf_func_convert(df_res2["label_click"],df_res2["time_min"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_res2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------+----------+---------+--------------------+---------+---------+
|             room_id|                 uid|label_click|watch_time|watch_cnt|            time_min|cvr_label|ctr_label|
+--------------------+--------------------+-----------+----------+---------+--------------------+---------+---------+
|b56a18e0eacdf51aa...|000025bbc8e3fd2a3...|          0|         0|        0|                 0.0|        0|        0|
|f3475ed497caeab39...|000026c67a83fa72a...|          0|         0|        0|                 0.0|        0|        0|
|89209f444391e9c72...|000048f2d4a90b363...|          0|    611792|       10|  10.196533333333333|        0|        0|
|5ec5fa5a5d586714f...|000069059776e2d3e...|          0|         0|        0|                 0.0|        0|        0|
|72204d87e2fec8e0e...|000073176e737404d...|          0|      2017|        1| 0.03361666666666667|        0|        0|
|ab38e6d41fbfd5519...|00011cf6740bf70c8...|          0| 

In [26]:
df_res2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- room_id: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- label_click: integer (nullable = true)
 |-- watch_time: long (nullable = true)
 |-- watch_cnt: integer (nullable = true)
 |-- time_min: double (nullable = false)
 |-- cvr_label: integer (nullable = true)
 |-- ctr_label: integer (nullable = true)

# process item 

In [27]:
#processs item feature
#cate_id
#child_id 
#tag_id
def regex_filter(x):
    try:
        a = list(json.loads(x).values())
    except:
        raise ("exception!")
    
    #return ['owner_exp','owner_uid','cate_id','age','sex','room_name',\
    #'owner_level','fans','child_id','cq','msgWords','tag_id','constellation']
    
    return a


#use age, owner_level,sex,fans
udf_func_age = udf(lambda x: regex_filter(x)[3],returnType=IntegerType())
udf_func_owner_level = udf(lambda x: regex_filter(x)[6],returnType=IntegerType())
udf_func_sex = udf(lambda x: regex_filter(x)[4],returnType=StringType())
udf_func_fans = udf(lambda x: regex_filter(x)[7],returnType=IntegerType())
udf_func_cateid = udf(lambda x: regex_filter(x)[2],returnType=StringType())
udf_func_childid = udf(lambda x: regex_filter(x)[8],returnType=StringType())
udf_func_tagid = udf(lambda x: regex_filter(x)[11],returnType=StringType())
udf_func_constellation = udf(lambda x: regex_filter(x)[12],returnType=StringType())

df_item = df_item.withColumn("item_age",udf_func_age(df_item["room_feature"])).\
withColumn("owner_level",udf_func_owner_level(df_item["room_feature"])).\
withColumn("item_sex",udf_func_sex(df_item["room_feature"])).\
withColumn("fans",udf_func_fans(df_item["room_feature"])).\
withColumn("cate_id",udf_func_cateid(df_item["room_feature"])).\
withColumn("child_id",udf_func_childid(df_item["room_feature"])).\
withColumn("tag_id",udf_func_tagid(df_item["room_feature"])).\
withColumn("constellation",udf_func_constellation(df_item["room_feature"]))

df_item = df_item.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#join result

columns_to_drop = ['room_feature']
df_item_res = df_item.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_item_res.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- room_id: string (nullable = true)
 |-- anchor_level: integer (nullable = true)
 |-- anchor_score: integer (nullable = true)
 |-- is_top200_anchor: integer (nullable = true)
 |-- room_click_num_1_day: integer (nullable = true)
 |-- room_click_num_7_day: integer (nullable = true)
 |-- room_click_num_15_day: integer (nullable = true)
 |-- room_ctr_1_day: double (nullable = false)
 |-- room_ctr_7_day: double (nullable = false)
 |-- room_ctr_15_day: double (nullable = false)
 |-- i_hot_mon: integer (nullable = true)
 |-- i_hot_fri: integer (nullable = true)
 |-- i_hot_weekend: integer (nullable = true)
 |-- i_hot_weekday: integer (nullable = true)
 |-- i_ctr_mon: double (nullable = false)
 |-- i_ctr_fri: double (nullable = false)
 |-- i_ctr_weekend: double (nullable = false)
 |-- i_ctr_weekday: double (nullable = false)
 |-- room_period_favor_30d: string (nullable = true)
 |-- guess_cnt_1d: integer (nullable = true)
 |-- guess_cnt_7d: integer (nullable = true)
 |-- guess_cnt_15d: 

# process user

In [30]:
#join result

df_user_res = df_user.select(['uid','phone_status','sex','province'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_user_res.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+---+--------+
|                 uid|phone_status|sex|province|
+--------------------+------------+---+--------+
|99f11d010f20a896f...|           1|  1|    贵州|
|a4c21e4414931098e...|           1|  1|    江西|
|c7db60a3d185f6e41...|           1|  0|    江苏|
|2cf3eafa09fe065ad...|           1|  0|    湖北|
|ddae618dcc52d31d2...|           1|  0|    null|
|d018b51c6fa552fb3...|           1|  0|    广东|
|e449870f68316ccf9...|           1|  1|    河南|
|9929519d78d9151ba...|           1|  0|  内蒙古|
|a6f1e4f0d8c02966d...|           1|  1|    山东|
|00b03cc7c3e1fd7a9...|           1|  0|    云南|
|555015100a2dcc692...|           1|  1|    陕西|
|3b33c683bfe87d365...|           1|  0|    广西|
|1f4ca56c9ae3e4848...|           1|  0|    河北|
|e91d5843ccad719d0...|           1|  1|    河北|
|0525d090e1cf1753a...|           0|  0|    江苏|
|c03861eef04571e5a...|           1|  0|    浙江|
|3ca4b234450b18e75...|           1|  0|    江西|
|28992ce4a681579c0...|           1|  1|    广东|
|54e94

# join results 

In [32]:
#join result

df_weight_item = df_item_res.join(df_res2.select(["uid","room_id","cvr_label","ctr_label"]), on="room_id", how="right")
df_res_all = df_weight_item.join(df_user_res,on='uid',how='left')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_res_all.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+------------+----------------+--------------------+--------------------+---------------------+--------------+--------------+---------------+---------+---------+-------------+-------------+---------+---------+-------------+-------------+---------------------+------------+------------+-------------+-------------+-------------+--------------+-------------------------+---------------------------+-------------------------+-------------------------+-------------------------+--------------------------+-------------------------+------------------------+--------+-----------+--------+----+-------+--------+------+-------------+---------+---------+------------+---+--------+
|                 uid|             room_id|anchor_level|anchor_score|is_top200_anchor|room_click_num_1_day|room_click_num_7_day|room_click_num_15_day|room_ctr_1_day|room_ctr_7_day|room_ctr_15_day|i_hot_mon|i_hot_fri|i_hot_weekend|i_hot_weekday|i_ctr_mon|i_ctr_fri|i_ctr_wee

# save results 

In [34]:

df_train, df_test = df_res_all.randomSplit([0.9, 0.1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
#save results

df_train.write.format('csv').\
    option('header', True).mode('overwrite').option('sep', '\t').\
    save('s3://douyu-datalab/train_1221_full.csv')

df_test.write.format('csv').\
    option('header', True).mode('overwrite').option('sep', '\t').\
    save('s3://douyu-datalab/test_1221_full.csv')

print("It take {:.2f} mimutes to finish".format((time.time()-Timer1)/60))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

It take 68.88 mimutes to finish